## Предметная область: Игра Dota 2

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) — многопользовательская компьютерная игра жанра [MOBA](https://ru.wikipedia.org/wiki/MOBA). Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

Существуют [разные режимы игры](http://dota2.gamepedia.com/Game_modes/ru), мы будем рассматривать режим [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

### Как проходит матч

#### 1. Игроки выбирают героев

Всего в игре чуть более 100 различных героев (персонажей). В начале игры, команды в определенном порядке выбирают героев себе и запрещают выбирать определенных героев противнику (баны). Каждый игрок будет управлять одним героем, в рамках одного матча не может быть несколько одинаковых героев.  Герои различаются между собой своими характеристиками и способностями. От комбинации выбранных героев во многом зависит успех команды.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Основная часть

Игроки могут получать золото и опыт за убийство чужих героев или прочих юнитов. Накопленный опыт влияет на уровень героя, который в свою очередь позволяет улучшать способности. За накопленное золото игроки покупают предметы, которые улучшают характеристики героев или дают им новые способности.

После смерти герой отправляется в "таверну" и возрождается только по прошествии некоторого времени, таким образом команда на некоторое время теряет игрока, однако игрок может досрочно выкупить героя из таверны за определенную сумму золота.

В течение игры команды развивают своих героев, обороняют свою часть поля и нападают на вражескую.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. Конец игры

Игра заканчивается, когда одна из команд разрушет определенное число "башен" противника и уничтожает трон.

![](http://imgur.com/Du79Kzf.jpg)

## Задача: предсказание победы по данным о первых 5 минутах игры

По первым 5 минутам игры предсказать, какая из команд победит: Radiant или Dire?     
`Метрика качества - площадь под ROC-кривой.`

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('features.csv', index_col="match_id")
test = pd.read_csv('features_test.csv', index_col="match_id")

#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

In [3]:
train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [4]:
test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [5]:
train.shape

(97230, 108)

> - Тренировочная выборка содержит признаки, которых нет в тестовой - это характеристики матча, которые не известны в первые пять минут (продолжительность матча, состояние башен к концу матча) - их нужно удалить.    
- Признак 'radiant_win' является таргетом.

#### Заполнение пропусков

In [6]:
train.isna().sum().sort_values(ascending=False)[:15]

first_blood_player2            43987
radiant_flying_courier_time    27479
dire_flying_courier_time       26098
first_blood_player1            19553
first_blood_team               19553
first_blood_time               19553
dire_bottle_time               16143
radiant_bottle_time            15691
radiant_first_ward_time         1836
dire_first_ward_time            1826
radiant_courier_time             692
dire_courier_time                676
r4_lh                              0
r4_kills                           0
r4_deaths                          0
dtype: int64

> Большинство пропусков находятся в категории first_blood. В описании признаков указано, что если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение. Поэтому вместо пропущенных значений укажем информацию об остутствии события.

In [7]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [8]:
target = train['radiant_win']

In [9]:
extra = set(train.columns) - set(test.columns) # найдем лишние колонки
features = train.drop(extra, axis=1) # и удалим их

### Градиентный бустинг

#### Gradient Boosting Classifier

In [10]:
cv = KFold(n_splits=5, shuffle=True)
best_score = 0
for est in [10, 20, 30]:
    GBclf = GradientBoostingClassifier(n_estimators=est, random_state=1)
    score = cross_val_score(GBclf, features, target, cv=cv, scoring='roc_auc').mean()
    if score > best_score:
        print("Кол-во деревьев", est, ":", score)
        best_score = score
        best_model_GB = GBclf

Кол-во деревьев 10 : 0.6651865606019264
Кол-во деревьев 20 : 0.682310514245471
Кол-во деревьев 30 : 0.6898931106775492


In [27]:
%%time
GBclf = GradientBoostingClassifier(n_estimators=30, random_state=1)
score = cross_val_score(GBclf, features, target, cv=cv, scoring='roc_auc').mean()

Wall time: 5min 59s


#### Cat Boosting Classifier

In [12]:
cv = KFold(shuffle=True)
cat_model = CatBoostClassifier(eval_metric='AUC')

score = cross_val_score(cat_model, features, target, cv=cv, scoring='roc_auc').mean()

Learning rate set to 0.066122
0:	total: 240ms	remaining: 3m 59s
1:	total: 366ms	remaining: 3m 2s
2:	total: 503ms	remaining: 2m 47s
3:	total: 629ms	remaining: 2m 36s
4:	total: 787ms	remaining: 2m 36s
5:	total: 939ms	remaining: 2m 35s
6:	total: 1.16s	remaining: 2m 44s
7:	total: 1.33s	remaining: 2m 44s
8:	total: 1.48s	remaining: 2m 42s
9:	total: 1.59s	remaining: 2m 37s
10:	total: 1.73s	remaining: 2m 35s
11:	total: 1.86s	remaining: 2m 33s
12:	total: 2s	remaining: 2m 31s
13:	total: 2.17s	remaining: 2m 32s
14:	total: 2.37s	remaining: 2m 35s
15:	total: 2.48s	remaining: 2m 32s
16:	total: 2.61s	remaining: 2m 31s
17:	total: 2.74s	remaining: 2m 29s
18:	total: 2.9s	remaining: 2m 29s
19:	total: 3.02s	remaining: 2m 28s
20:	total: 3.18s	remaining: 2m 28s
21:	total: 3.36s	remaining: 2m 29s
22:	total: 3.53s	remaining: 2m 30s
23:	total: 3.64s	remaining: 2m 28s
24:	total: 3.78s	remaining: 2m 27s
25:	total: 3.92s	remaining: 2m 26s
26:	total: 4.05s	remaining: 2m 26s
27:	total: 4.18s	remaining: 2m 25s
28:	t

In [13]:
score

0.7234372617895846

### Первые выводы    
     
1. Для обучения моделей градиентного бустинга понадобилось избавиться от пропусков. Некоторые из них несли смысл - они обозначали не отсутвие информации об игре (как обычно), а отсутвие самого события. Например признак first_blood_team обозначает команду, совершивую "первую кровь", а пропуск в этом признаке - ни одна команда не сделала это в первые пять минут. Все остальные признаки категории first_blood имеют тот же смысл.   
   
  Если заменить пропуски first_blood на очень большое или маленькое число, то модель будет показывать такое же качество, как и при замене на 0, а обучаться намного дольше (выяснилось опытным путем). Поэтому все пропуски были заменены на 0. В данных также были лишние признаки, содержащие информацию, неизвестную в первые пять минут игры. В каком-то смысле эти признаки - утечка данных, поэтому для обучения они не нужны, и мы их удалили.       
           
           
2. Для обучения из данных был извлечен столбец radiant_win, являющийся целевой переменной.   

3-4. Модель Gradient Boosting Classifier показала результат roc_auc = 0,69 при 30 деревьях, при этом качество прогноза увеличивалось при увеличении числа деревьев. Обучение на 30 деревьях длилось 6 минут. Для того чтобы ускорить обучение при большем количестве деревьев, можно отрегулировать другие гиперпараметры. Например, ограничить глубину деревьев или увеличить шаг.

Для сравнения была также обучена модель CatBoost. Её обучение занимает больше времени, но дает лучший результат. Ее метрика ROC AUC равна 72. 

### Логистическая регрессия

In [14]:
def log_regr(features, target):
    """
    Функция принимает на вход данные,
    масштабирует их, строит модель логистической регрессии
    и иллюстрирует качество модели при заданном параметре регуляризации
    """
    
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    cv = KFold(shuffle=True)
    for c in np.linspace(1, 5, 10): # подбор параметра C
        log_reg = LogisticRegression(max_iter=100, C=c)
        log_reg.fit(features_scaled, target)
        score = cross_val_score(log_reg, features_scaled, target, cv=cv, scoring='roc_auc').mean()
        print('C:', round(c, 2), '|', 'Score:', round(score, 2))

In [17]:
log_regr(features, target)

C: 1.0 | Score: 0.72
C: 1.44 | Score: 0.72
C: 1.89 | Score: 0.72
C: 2.33 | Score: 0.72
C: 2.78 | Score: 0.72
C: 3.22 | Score: 0.72
C: 3.67 | Score: 0.72
C: 4.11 | Score: 0.72
C: 4.56 | Score: 0.72
C: 5.0 | Score: 0.72


In [24]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [25]:
%%time

log_reg = LogisticRegression(max_iter=100)
log_reg.fit(features_scaled, target)

Wall time: 2.06 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

> Логистическая регрессия работает гораздо быстрее градиентоного бустинга и дает лучший результат - 0,72 ROC AUC за 2 секунды. Однако, отставание градиентного бустинга скорее всего объясняется нетщательным подбором паратеров. Более качественная оптимизация параметров при построении модели бустинга дала бы ей больший результат.

### Удаление категориальных признаков

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [19]:
features

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,0,-5.0,0.0,-82.0,0.0,4,3,2,0,-17.0
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,2,-32.0,249.0,-70.0,0.0,1,1,3,1,-15.0


In [20]:
categorical = ["lobby_type", "r1_hero", "r2_hero", "r3_hero", "r4_hero", "r5_hero", 
              "d1_hero", "d2_hero", "d3_hero", "d4_hero", "d5_hero"]

In [21]:
features_without_heros = features.drop(categorical, axis=1)

In [22]:
log_regr(features_without_heros, target)

C: 1.0 | Score: 0.72
C: 1.44 | Score: 0.72
C: 1.89 | Score: 0.72
C: 2.33 | Score: 0.72
C: 2.78 | Score: 0.72
C: 3.22 | Score: 0.72
C: 3.67 | Score: 0.72
C: 4.11 | Score: 0.72
C: 4.56 | Score: 0.72
C: 5.0 | Score: 0.72


> Качество модели совсем не изменилось. Это связано с тем, что категориальные (хоть и выраженные числами) признаки требуют кодирования, иначе модель воспринимает их как количественные и считает, что большее число, обозначающее одну группу объектов важнее меньшего, обозначающего другую группу. Для наших данных это неверный подход и модель не извлекла нужной информации из категориальных признаков, поэтому дала этим признакам малый вес.

#### Количество героев

На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре.

In [28]:
len(features.r1_hero.unique())

108

In [29]:
max(features.r1_hero.unique())

112

> В данных содержится информация о 108 игроках, при этом максимальный инедекс игрока - 112. Это значит, что согласно нашим данным 4 игрока в матчах не ипользуется вообще. Возможны три варианта: герои слабые и непопулярные или постоянно забанены противником из-за того что очень сильные, или с выборкой что-то не так.

#### Кодирование с помощью мешка слов

In [30]:
#  к од для формирования "мешка слов" по героям

def encoding(data):
    X_pick = np.zeros((data.shape[0], 112))
    for i, match_id in enumerate(data.index):
        for p in range(5):
            r_hero = 'r%d_hero' % (p+1)
            d_hero = 'd%d_hero' % (p+1)

            hero_name_r = data.loc[match_id, r_hero]
            hero_name_d = data.loc[match_id, d_hero]

            X_pick[i, data.loc[match_id, r_hero]-1] = 1
            X_pick[i, data.loc[match_id, d_hero]-1] = -1
    return X_pick

In [33]:
X_pick = encoding(features)

In [34]:
features_scaled_categorized = np.hstack((features_without_heros, X_pick))

In [35]:
features_scaled_categorized

array([[-2.54436416,  1.40080818,  1.52597175, ...,  0.        ,
         0.        ,  0.        ],
       [-2.54045236,  0.50131354, -0.08013929, ...,  0.        ,
         0.        ,  0.        ],
       [-2.53923104,  0.50131354,  0.1510701 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.09874571,  0.50131354,  0.29226667, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.09895204, -0.39818111, -0.17368203, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.1026479 , -0.39818111, -0.31840851, ...,  0.        ,
         0.        , -1.        ]])

In [36]:
log_regr(features_scaled_categorized, target)

C: 1.0 | Score: 0.75
C: 1.44 | Score: 0.75
C: 1.89 | Score: 0.75
C: 2.33 | Score: 0.75
C: 2.78 | Score: 0.75
C: 3.22 | Score: 0.75
C: 3.67 | Score: 0.75
C: 4.11 | Score: 0.75
C: 4.56 | Score: 0.75
C: 5.0 | Score: 0.75


> Качество модели значительно улучшилось: кодирование и масштабирование признаков помогли модели справиться с прогнозированием исхода матча.

### Прогнозирование на тестовой выборке

In [37]:
X_pick_test = encoding(test) 
test_scaled_and_categorized = np.hstack((scaler.transform(test.drop(categorical, axis=1)), 
                                                  X_pick_test))

In [38]:
log_reg = LogisticRegression()
log_reg.fit(features_scaled_categorized, target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
predictions = log_reg.predict(test_scaled_and_categorized)

In [40]:
predictions_proba = log_reg.predict_proba(test_scaled_and_categorized)[:, 1]

In [41]:
print('Минимальное значение прогноза:', round(min(predictions_proba), 4))
print('Максимальное значение прогноза:', round(max(predictions_proba), 4))

Минимальное значение прогноза: 0.0085
Максимальное значение прогноза: 0.9964


Задача классификации решалась путем построения моделей градиентного бустинга и логистической регресии. На начальном этапе модели показывали идентичный результат, с той лишь разницей, что логистическая регрессия справлялась с обучением гораздо быстрее. После обработки данных: масштабирования количественнных признаков и кодирования категориальных ROC AUC лог.регрессии стал еще выше; значение параметра регуляризации не меняло модели из-за того, что данных очень много и модель не переобучается, поэтому регуляризация бесполезна. Предсказание строилось с помощью модели логистической регрессии.